In [2]:
import pandas as pd

def load_shots_data(player_name):
    file_path = f"../../data/mens/{player_name}/combined.xlsx"
    xls = pd.ExcelFile(file_path)
    return xls.parse('Shots')

def best_return_strokes(data, player_name):
    returns = data[(data['Player'] == player_name) & (data['Shot'].isin([2, 3]))].copy()
    returns = returns[returns['Stroke'].isin(['Forehand', 'Backhand'])]
    returns['ReturnType'] = returns['Stroke']
    last_shots = (
        data.sort_values(['__source_file__', 'Point', 'Shot'])
        .groupby(['__source_file__', 'Point'])
        .last()
        .reset_index()
        [['__source_file__', 'Point', 'Player']]
        .rename(columns={'Player': 'PointWinner'})
    )
    returns = returns.merge(last_shots, on=['__source_file__', 'Point'], how='left')
    returns['PointWon'] = returns['PointWinner'] == player_name
    stroke_labels = {2: 'First Return', 3: 'Second Return'}
    returns['ReturnStroke'] = returns['Shot'].map(stroke_labels)
    summary = (
        returns.groupby(['ReturnStroke', 'ReturnType'])['PointWon']
        .sum()
        .reset_index()
        .rename(columns={'PointWon': 'PointsWon'})
        .sort_values(by='PointsWon', ascending=False)
    )
    return summary

# Load and run
player_name = "Rudy Quan"
shots_df = load_shots_data(player_name)
summary_df_labeled = best_return_strokes(shots_df, player_name)
print(summary_df_labeled)

    ReturnStroke ReturnType  PointsWon
0   First Return   Backhand        204
3  Second Return   Forehand        200
1   First Return   Forehand        168
2  Second Return   Backhand         85
